<a href="https://colab.research.google.com/github/dayviddouglas/ciencia_de_dados_licitacoes_prefeitura_do_recife/blob/test/dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dash
!pip install plotly-express

In [2]:
import pandas as pd
import requests as req
from io import StringIO
import urllib
import matplotlib.pyplot as plt
import plotly.subplots as sp
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import dash as dsh
from dash import Dash, html,dcc,Input,Output
import plotly.express as px

file_id= "1-0_rPre-v9jjAg1hwdK_mbfYUQqfCvYS"
url =  f"https://drive.google.com/uc?id={file_id}"


try:
    response = req.get(url)
    response.encoding = 'utf-8'
    response.raise_for_status()  # Lança um erro para respostas não-sucedidas
    # Usando StringIO para converter o texto em um arquivo em memória e, então, lendo com o Pandas
    csv_raw = StringIO(response.text)
    base = pd.read_csv(csv_raw, encoding='utf-8')



except req.RequestException as e:
    print(f"Erro ao acessar o arquivo: {e}")




In [ ]:
base

In [ ]:
base2 = base.copy()

def abreviar_coluna_especifica(base2, coluna_especifica):
    # Verifica se a coluna especificada existe no DataFrame
    if coluna_especifica in base2.columns:
        # Aplica a função de abreviação somente na coluna especificada
        base2[coluna_especifica] = base2[coluna_especifica].apply(lambda x: abreviar_apos_primeiro_nome(x) if isinstance(x, str) else x)
    return base2

def abreviar_apos_primeiro_nome(s):
    partes = s.split(maxsplit=1)
    # Mantém o primeiro nome e abrevia o restante das palavras em grupos de 3 caracteres
    if len(partes) > 1:
        # Abrevia cada palavra do restante da string em grupos de 3 caracteres
        abreviacao = ' '.join([palavra[:3] for palavra in partes[1].split()])
        return partes[0] + ' ' + abreviacao
    else:
        return s

# Supondo que base2 seja um DataFrame do pandas, a função agora será aplicada somente na coluna 'orgao_licitante'.
# O código a seguir é apenas para demonstração e não será executado aqui.
base2['orgao_abreviado'] = base2['orgao_licitante']
base2['comissao_abreviado'] = base2['comissao_licitacao']
base2['objeto_abreviado'] = base2['objeto']

base2 = abreviar_coluna_especifica(base2, 'orgao_abreviado')
base2 = abreviar_coluna_especifica(base2, 'comissao_abreviado')
base2 = abreviar_coluna_especifica(base2, 'objeto_abreviado')
base2.head()

<h2> Criando Dataframe de Processos Licitatórios por Estado.</h2>

In [ ]:
process_uf= pd.DataFrame(base["uf_endcontratado"].value_counts())
process_uf.reset_index(inplace=True)
process_uf.columns = ['uf_endcontratado', 'count']
cinco_mais_estados=process_uf.head(5)





<h2> Criando gráfico de licitações por estado.</h2>

In [ ]:
# graf_process_uf= go.Figure(data=[go.Pie(labels=cinco_mais_estados['uf_endcontratado'], values=cinco_mais_estados['count'], name="Licitações por estado")])
# graf_process_uf



In [5]:

process_pe=base[base["uf_endcontratado"]== "PE"]
process_pe["cidade_endcontratado"].value_counts()

# Criando o dataframe Quantidade de processos por cidade

df_process_cidade = pd.DataFrame(process_pe["cidade_endcontratado"].value_counts())
df_process_cidade.reset_index(inplace=True)
df_process_cidade.columns = ['cidade_endcontratado', 'count']






In [6]:
# Transformando a filtragem cidades cujos os produtos ou serviços participaram de licitações em dataframe.
df_process_cidade = pd.DataFrame(process_pe["cidade_endcontratado"].value_counts())
df_process_cidade.reset_index(inplace=True)
df_process_cidade.columns = ['cidade_endcontratado', 'count']
cincos_cidades_com_mais_process=df_process_cidade.iloc[0:5]


In [ ]:
# Criando o gráfico dos processos por cidade.
graf_process_cidade2 = go.Figure(data=[go.Pie(labels=cincos_cidades_com_mais_process['cidade_endcontratado'], values=cincos_cidades_com_mais_process['count'], name="Licitações por cidade")])
graf_process_cidade2

In [ ]:
# Transformando a filtragem sobre as 5 comissões que mais participaram de solicitações em dataframe.
comissoes_licitantes= pd.DataFrame(base2["comissao_abreviado"].value_counts())
comissoes_licitantes.reset_index(inplace=True)
comissoes_licitantes.columns = ['comissao_licitacao', 'count']
comissoes_value=[
    "CPLMSA",
    "CPLSSA",
    "CPLM",
    "EQUIPE DE PREGÃO - EMLURB",
    "COMISSÃO DE LICITAÇÃO - EMLURB"
]
cinco_mais_comissoes=comissoes_licitantes.iloc[0:5]
cinco_mais_comissoes['comissao_licitacao'] = comissoes_value
cinco_mais_comissoes



In [9]:
# Criando gráfico das 5 comissões que mais participaram de licitações.
graf_comissoes_licitantes=px.bar(cinco_mais_comissoes, x="comissao_licitacao", y="count", labels={
                     "comissao_licitacao": "Comissão",
                     "count": "Quantidade"}, barmode="group", title="Quantidade de processos por comissão.")

In [ ]:
freq_orgao_licitantes=pd.DataFrame(base2['orgao_abreviado'].value_counts())
freq_orgao_licitantes.reset_index(inplace=True)
freq_orgao_licitantes.columns = ['orgao_licitante', 'count']
orgao_values = [
    "FUNDO MUN. DE SAÚDE",
    "EMLURB",
    "FUNDAÇÃO DE CULTURA",
    "SEC. DE EDUCAÇÃO",
    "FMAS"
]
cinco_mais_orgaos=freq_orgao_licitantes.iloc[0:5]
cinco_mais_orgaos['orgao_licitante'] = orgao_values
cinco_mais_orgaos

In [11]:
# Criando gráfico dos 5 orgãos que mais participaram de licitações.
graf_orgao_licitantes=px.bar(cinco_mais_orgaos, x="orgao_licitante", y="count", labels={
                     "orgao_licitante": "Orgão",
                     "count": "Quantidade"}, barmode="group", title="Quantidade de processos por orgão.")

In [12]:
# Transformando a filtragem sobre os 5 objetos que mais participaram de solicitações em dataframe.
freq_objeto=pd.DataFrame(base2["objeto_abreviado"].value_counts().head(5))
freq_objeto.reset_index(inplace=True)
freq_objeto.columns = ['objeto', 'count']

objeto_values = [
    "AQUISICAO DE MATERIAL HOSPITALAR",
    "AQUISIÇÃO DE MEDICAMENTOS",
    "REGISTRO DE PREÇO DE MATERIAL HOSPITALAR",
    "REGISTRO DE PREÇO DE MATERIAIS ELÉTRICOS",
    "REGISTRO DE PREÇOS PARA AQUISIÇÃO DE MEDICAMENTOS"
]


freq_objeto['objeto'] = objeto_values
freq_objeto['contador'] = freq_objeto.index

In [13]:
# Criando o Gráfico de objetos que mais participaram de licitações.
graf_objeto=px.bar(freq_objeto, x="objeto", y="count", labels={
                     "objeto": "Objeto",
                     "count": "Quantidade"}, barmode="group", title="Quantidade de licitações por objeto.")

In [14]:
# Transformando a filtragem sobre os 5 valores homologados que mais participaram de solicitações em dataframe.
freq_val_homologados=pd.DataFrame(base["valor_totalhomologadolicitacao"].value_counts().head(5))
freq_val_homologados.reset_index(inplace=True)
freq_val_homologados.columns = ['valor_totalhomologadolicitacao', 'count']



In [ ]:
# Criando o Gráfico de valores Homologados que mais participaram de licitações.
graf_valor_homolog = go.Figure(data=[go.Bar(x=freq_val_homologados["valor_totalhomologadolicitacao"], y=freq_val_homologados["count"], name='Quantidade de licitações homologadas')])
graf_valor_homolog.update_layout(title='Frequência de valores homologados', xaxis_title='Valor homologado', yaxis_title='Quantidade',template="plotly_dark")





In [16]:
# Transformando a filtragem sobre os 5 valores por lote que mais participaram de licitações em dataframe.
freq_val_lote=pd.DataFrame(base["valor_licitacao_lote"].value_counts().head(5))
freq_val_lote.reset_index(inplace=True)
freq_val_lote.columns = ['valor_licitacao_lote', 'count']



In [ ]:
# Criando o Gráfico de licitações por lote.
graf_valor_lote = go.Figure(data=[go.Bar(x=freq_val_lote["valor_licitacao_lote"], y=freq_val_lote["count"], name='Quantidade de licitações por lote')])


graf_valor_lote.update_layout(title='Frequência de valores de licitação por lote', xaxis_title='Valor por lote', yaxis_title='Quantidade',template="plotly_dark")

In [18]:
lista_grafico = [

             {'id_grafico':'gráfico-processos-pe-por-cidade','df':cincos_cidades_com_mais_process, 'x_grafico':'cidade_endcontratado',
             'y_grafico':'count', 'label1':'Cidade', 'label2':'Quantidade','titulo':"Quantidade de processos por cidade."},

            {'id_grafico':'gráfico-comissao-licitacao','df':cinco_mais_comissoes, 'x_grafico':'comissao_licitacao',
             'y_grafico':'count', 'label1':'Comissão', 'label2':'Quantidade','titulo':"Quantidade de processos por comissão."},

            {'id_grafico':'gráfico-orgao-licitante','df':cinco_mais_orgaos,'x_grafico':'orgao_licitante','y_grafico':'count',
             'label1':'Orgão','label2':'Quantidade','titulo':"Quantidade de processos por orgão."},

             {'id_grafico':'gráfico-objeto','df':freq_objeto,'x_grafico':'objeto',
             'y_grafico':'count', 'label1':'objeto', 'label2':'Quantidade','titulo':"Quantidade de processos por objeto"},
           ]

In [ ]:
from binascii import hexlify
app= Dash(__name__)

opcoes=[{'label': 'Processos de Licitação em Pernambuco', 'value': 'gráfico-processos-pe-por-cidade'},
        {'label': 'Comissões que participaram de Licitação', 'value': 'gráfico-comissao-licitacao'},
        {'label': 'Orgãos que participaram de licitação', 'value': 'gráfico-orgao-licitante'},
        {'label': 'Objetos que participaram de licitação', 'value': 'gráfico-objeto'},]



opcoes.append({'label': 'Todos os gráficos', 'value': 'todos_os_graficos'})


app.layout=html.Div(children=[

        html.Div([
            html.H1(id="texto_principal", children="Processos de Licitação no Estado de Pernambuco"),
        html.A([
      html.Button("analise de dados",style={'border':'none','border':'solid','cursor':'pointer','color':'white','padding':'1rem','backgroundColor':'rgb( 63 63 70)'}),
   ],href="#grafico_s",target="_self"),
    html.A([
        html.Button("valores por lote",style={'border':'none','border':'solid','cursor':'pointer','color':'white','padding':'1rem','backgroundColor':'rgb( 63 63 70)'}),
    ],href="#valor_lote",target="_self"),
    html.A([
        html.Button("valores homologados",style={'border':'none','border':'solid','cursor':'pointer','color':'white','padding':'1rem','backgroundColor':'rgb( 63 63 70)'}),
    ],href="#valor_homolog",target="_self"),
    ],style={'display':'flex','flex':'column','alignItems':'center','justifyContent':'space-around'}),
    html.Div([
      html.H3(children="O presente dashboard faz uma análise dos processos licitatórios, visto que as licitações ocorrem em sua maioria no seu próprio estado."),
      html.H3(children="Foram abordados os seguintes pontos: a quantidade de licitações por cidades, comissão, orgãos e objetos."),
    ],style={'textAlign':'center','marginTop':'5rem'}),

    dcc.Dropdown(opcoes,'todos_os_graficos', id='drop_down',style={'width':'50vw','color':'black','margin':'auto','marginTop':'5rem'}),

        dcc.Graph(id= "grafico_s"),

        html.H3(children="Outro tópico abordado, foi os processos licitátorios por lote, observando a frequência foi constatado os valores que tiveram maior constância sendo assim, entende-se os padrões de comportamento das licitações por lote",style={'textAlign':'center','marginTop':'5rem'}),

        dcc.Graph(id="valor_lote",figure=graf_valor_lote),

        html.H3(children="Análise estátistica dos válores por lote homologados:"),

        html.Div([html.P(children="Moda:"+str(base['valor_licitacao_lote'].mode()[0]),style={'border':'solid','borderRadius':'15px','padding':'1rem'}),

        html.P(children="Média:"+str(base['valor_licitacao_lote'].mean()),style={'border':'solid','borderRadius':'15px','padding':'1rem'}),

        html.P(children="Mediana:"+str(base['valor_licitacao_lote'].median()),style={'border':'solid','borderRadius':'15px','padding':'1rem'}),

        html.P(children="Desvio Padrão:"+str(base['valor_licitacao_lote'].std()),style={'border':'solid','borderRadius':'15px','padding':'1rem'}),

        html.P(children="Variância:"+str(base['valor_licitacao_lote'].var()),style={'border':'solid','borderRadius':'15px','padding':'1rem'}),

        html.Div([html.P(children="Assimetria: Positiva, indicando uma cauda mais longa ou mais espessa à direita."),
        html.P(children="Assimetria:"+str(base['valor_licitacao_lote'].skew()))],style={'border':'solid','borderRadius':'15px','padding':'1rem'}),


        html.Div([html.P(children="Curtose: Leptocúrtica,pois tem um excesso de curtose positiva."),
        html.P(children="Curtose:"+str(base['valor_licitacao_lote'].kurtosis()))],style={'border':'solid','borderRadius':'15px','padding':'1rem'}),

        ],style={'display':'flex','flexDirection':'row','flexWrap':'wrap','justifyContent':'center','alignItems':'center','gap':'1rem'}),


        html.H3(children="Outro ponto foram os valores totais homologados onde foi analisado que a grande maioria dos valores que foram homologados tinham seus valores entre 47 E 48 milhões de reais.",style={'textAlign':'center','marginTop':'8rem'}),

        dcc.Graph(id="valor_homolog",figure=graf_valor_homolog),

        html.H3(children="Análise estatistica dos valores totais homologados:"),

        html.Div([


        html.P(children="Moda:"+str(base['valor_totalhomologadolicitacao'].mode()[0]),style={'border':'solid','borderRadius':'15px','padding':'1rem'}),

        html.P(children="Média:"+str(base['valor_totalhomologadolicitacao'].mean()),style={'border':'solid','borderRadius':'15px','padding':'1rem'}),

        html.P(children="Mediana:"+str(base['valor_totalhomologadolicitacao'].median()),style={'border':'solid','borderRadius':'15px','padding':'1rem'}),

        html.P(children="Desvio Padrão:"+str(base['valor_totalhomologadolicitacao'].std()),style={'border':'solid','borderRadius':'15px','padding':'1rem'}),

        html.P(children="Variância:"+str(base['valor_totalhomologadolicitacao'].var()),style={'border':'solid','borderRadius':'15px','padding':'1rem'}),

        html.Div(
            [
        html.P(children="Assimetria: Positiva, indicando uma cauda mais longa ou mais espessa à direita."),
        html.P(children="Assimetria:"+str(base['valor_totalhomologadolicitacao'].skew()))
        ],style={'border':'solid','borderRadius':'15px','padding':'1rem'}
                 ),

        html.Div([
        html.P(children="Curtose: Leptocúrtica,pois tem um excesso de curtose positiva."),
        html.P(children="Curtose:"+str(base['valor_totalhomologadolicitacao'].kurtosis()))]
                 ,style={'border':'solid','borderRadius':'15px','padding':'1rem'})
        ],style={'display':'flex','flexDirection':'row','flexWrap':'wrap','justifyContent':'center','alignItems':'center','gap':'1rem','paddingBottom':'2rem'}

                 ),


],style={'color':'white','backgroundColor':'rgb(24 24 27)'})


@app.callback(
    Output('grafico_s', 'figure'),
    Input('drop_down', 'value')
)
def update_output(value):
    if value == 'todos_os_graficos':
        specs = [[{'type':'domain'},{'type':'domain'}],
                [{'type':'bar'}, {'type':'bar'}],
                [{"colspan": 2}, None]]

        fig = make_subplots(rows=3, cols=2, specs=specs, horizontal_spacing=0.25,subplot_titles=("Porcentagem de processos por Estado",
                                                                                                 "Porcentagem de processos por cidade",
                                                                                                "Quantidade de processos por comissão",
                                                                         "Quantidade de processos por orgão",
                                                                         "Quantidade de processos por objeto"))
        fig.add_trace(go.Pie(labels=cinco_mais_estados['uf_endcontratado'],
                             textinfo='label+percent',
                             insidetextorientation='radial',
                             values=cinco_mais_estados['count'],
                             marker=dict(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'],
                             line=dict(color='#000000', width=2))),
                             row=1,col=1)


        fig.add_trace(go.Pie(labels=cincos_cidades_com_mais_process['cidade_endcontratado'],
                             textinfo='label+percent',
                             insidetextorientation='radial',
                             values=cincos_cidades_com_mais_process['count'],
                             marker=dict(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'],
                                         line=dict(color='#000000', width=2))),
                      row=1, col=2)

        # Add horizontal Bar charts with reversed x-axis
        fig.add_trace(go.Bar(x=cinco_mais_comissoes['count'],
                             y=cinco_mais_comissoes['comissao_licitacao'],
                             orientation='h',
                             name="Quantidade de processos por comissão"),
                      row=2, col=1)

        fig.add_trace(go.Bar(x=cinco_mais_orgaos['count'],
                             y=cinco_mais_orgaos['orgao_licitante'],
                             orientation='h',
                             name="Quantidade de processos por orgão"),
                      row=2, col=2)

        fig.add_trace(go.Bar(x=freq_objeto['count'],
                             y=freq_objeto['objeto'],
                             orientation='h',
                             name="Quantidade de processos por objeto"),
                      row=3, col=1)

        # Update layout with reversed x-axes for bar charts and move y-axis to the right
        fig.update_layout(showlegend=False,
                          title_text="Todos os gráficos",
                          template="plotly_dark",
                          height=900,
                          xaxis1=dict(autorange='reversed'),
                          xaxis2=dict(autorange='reversed'),
                          xaxis3=dict(autorange='reversed'),
                          xaxis4=dict(autorange='reversed'),
                          xaxis5=dict(autorange='reversed'),
                          yaxis1=dict(side='right'),
                          yaxis2=dict(side='right'),
                          yaxis3=dict(side='right'),
                          yaxis4=dict(side='right'),
                          yaxis5=dict(side='right')
                          )


        return fig

    else:
        for i in lista_grafico:
            if i['id_grafico'] == value:
                fig = go.Figure(data=[
                    go.Bar(x=i['df'][i['x_grafico']], y=i['df'][i['y_grafico']],
                           name="Quantidade de processos por cidade")
                ])
                fig.update_layout(title=i['titulo'],
                                  xaxis_title=i['label1'],
                                  yaxis_title=i['label2'],
                                  template="plotly_dark",
                                  height=400,
                                  )
                return fig


if __name__=='__main__':
    app.run(jupyter_mode="external")